In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances
import random

In [15]:
articles = pd.read_csv('articlesDataset/cleanDataArticles', delimiter=',', header=None)
tables = pd.read_csv('articlesDataset/cleanDataTables', delimiter=',', header=None)

In [3]:
formattedTables = tables.iloc[:,:].values
formattedArticles = articles.iloc[:,:].values

In [4]:
corpusTablePgTitle = []
corpusTableSectionTitle = []
corpusTableCaption = []
corpusTableHeader  = []
corpusTableBody = []

for tables in formattedTables:
    
    corpusTablePgTitle.append(str(tables[1]))
    corpusTableSectionTitle.append(str(tables[2]))
    corpusTableCaption.append(str(tables[3]))
    corpusTableHeader.append(str(tables[4]))
    corpusTableBody.append(str(tables[5]))

In [6]:
vectorizerPgTitle = TfidfVectorizer(stop_words='english', analyzer='word')
vectorizerSectionTitle = TfidfVectorizer(stop_words='english', analyzer='word')
vectorizerCaption = TfidfVectorizer(stop_words='english', analyzer='word')
vectorizerHeader = TfidfVectorizer(stop_words='english', analyzer='word')
vectorizerBody = TfidfVectorizer(stop_words='english', analyzer='word')

tablesVectorPgTitle = vectorizerPgTitle.fit_transform(corpusTablePgTitle)
tablesVectorSectionTitle = vectorizerSectionTitle.fit_transform(corpusTableSectionTitle)
tablesVectorCaption = vectorizerCaption.fit_transform(corpusTableCaption)
tablesVectorHeader = vectorizerHeader.fit_transform(corpusTableHeader)
tablesVectorBody = vectorizerBody.fit_transform(corpusTableBody)

In [ ]:
totalArticlesToSearch = 100

def selectRandomArticles():
    
    for i in range(totalArticlesToSearch):
    
        randomArticle = random.choice(formattedArticles)
    
        corpusArticlesID.append(randomArticle[0])
        corpusArticlesText.append(str(randomArticle[1]))

In [8]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [9]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
AverageTop5 = []
AverageTop10 = []
AverageTop15 = []
AverageTop20 = []

In [ ]:
def saveAccuracy(k,mean):
    
    if k == 5:
            
        AverageTop5.append(np.mean(accuracy))
        
    if k == 10:
            
        AverageTop10.append(np.mean(accuracy))
        
    if k == 15:
            
        AverageTop15.append(np.mean(accuracy))
        
    if k == 20:
            
        AverageTop20.append(np.mean(accuracy))

In [13]:
for execution in tqdm(range(30)):
    
    #print("execution = "+ str(execution))
    
    corpusArticlesID = []
    corpusArticlesText = []
    
    selectRandomArticles()
    
    #print(corpusArticlesID)
    
    articlesVectorPgTitle = vectorizerPgTitle.transform(corpusArticlesText)
    articlesVectorSectionTitle = vectorizerSectionTitle.transform(corpusArticlesText)
    articlesVectorCaption = vectorizerCaption.transform(corpusArticlesText)
    articlesVectorHeader = vectorizerHeader.transform(corpusArticlesText)
    articlesVectorBody = vectorizerBody.transform(corpusArticlesText)
    
    for accuracyK in range(5,21,5):
    
        accuracy = []

        totalSelectedArticles = int(articlesVectorPgTitle.shape[0])

        for i in range(totalSelectedArticles):
    
            distanceVectorPgTitle = pairwise_distances(articlesVectorPgTitle[i], tablesVectorPgTitle, metric='cosine')
            distanceVectorSectionTitle = pairwise_distances(articlesVectorSectionTitle[i], tablesVectorSectionTitle, metric='cosine')
            distanceVectorCaption = pairwise_distances(articlesVectorCaption[i], tablesVectorCaption, metric='cosine')
            distanceVectorHeader = pairwise_distances(articlesVectorHeader[i], tablesVectorHeader, metric='cosine')
            distanceVectorBody = pairwise_distances(articlesVectorBody[i], tablesVectorBody, metric='cosine')
    
            distanceVector = np.amax([distanceVectorPgTitle, distanceVectorSectionTitle, distanceVectorCaption, distanceVectorHeader, distanceVectorBody], axis=0)
            #distanceAvg = np.mean([distanceVectorCaption, distanceVectorHeader, distanceVectorText], axis=0)
            #distanceMax = np.amax([distanceVectorCaption, distanceVectorHeader, distanceVectorText], axis=0)
    
            idQueryGoal = int(corpusArticlesID[i])
    
            countTopTables = accuracyK
        
            topKRank = nsmallest(countTopTables, distanceVector[0])
    
            idRankedTables = getIdRankedTalbes(topKRank,distanceVector[0])
    
            accuracy.append(getAccuracy(idRankedTables,idQueryGoal))
    
        #save the accuracy on the list
        saveAccuracy(accuracyK,np.mean(accuracy))

100%|██████████| 30/30 [06:31<00:00, 12.93s/it]


In [14]:
print(str(round(np.mean(AverageTop5),4))+" (±) "+str(round(np.std(AverageTop5),4)))
print(str(round(np.mean(AverageTop10),4))+" (±) "+str(round(np.std(AverageTop10),4)))
print(str(round(np.mean(AverageTop15),4))+" (±) "+str(round(np.std(AverageTop15),4)))
print(str(round(np.mean(AverageTop20),4))+" (±) "+str(round(np.std(AverageTop20),4)))

0.0133 (±) 0.034
0.0233 (±) 0.0423
0.0267 (±) 0.0442
0.03 (±) 0.0458
